In [1]:
from transformers import AutoTokenizer, AutoModel
# tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/LLM/TinyLlama-1.1B-Chat-v1.0", trust_remote_code=True)
model = AutoModel.from_pretrained("/Users/tianlimin/chatglm3-6b", trust_remote_code=True)
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)


/Users/tianlimin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards:  57%|██████████▎       | 4/7 [00:04<00:03,  1.02s/it]


KeyboardInterrupt: 

In [ ]:
pip install torch torchvision torchaudio cudatoolkit=11.3 -c pytorch


In [ ]:
pip install protobuf transformers==4.30.2 cpm_kernels torch>=2.0 gradio mdtex2html sentencepiece accelerate


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/Users/tianlimin/chatglm3-6b", trust_remote_code=True)
model = model.eval()
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)


In [ ]:
#import gradio as gr
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import StoppingCriteria, StoppingCriteriaList, TextIteratorStreamer
from threading import Thread



In [ ]:
pip install git+https://github.com/huggingface/transformers

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/chatglm3-6b")
model = AutoModelForCausalLM.from_pretrained("/Users/tianlimin/chatglm3-6b")

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        stop_ids = [2]  # IDs of tokens where the generation should stop.
        for stop_id in stop_ids:
            if input_ids[0][-1] == stop_id:  # Checking if the last generated token is a stop token.
                return True
        return False


# Function to generate model predictions.
def predict(message, history):
    history_transformer_format = history + [[message, ""]]
    #stop = StopOnTokens()

    # Formatting the input for the model.
    messages = "</s>".join(["</s>".join(["\n<|user|>:" + item[0], "\n<|assistant|>:" + item[1]])
                        for item in history_transformer_format])
    model_inputs = tokenizer([messages], return_tensors="pt").to(device)
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)
    generate_kwargs = dict(
        model_inputs,
        streamer=streamer,
        max_new_tokens=1024,
        do_sample=True,
        top_p=0.95,
        top_k=50,
        temperature=0.7,
        num_beams=1,
        #stopping_criteria=StoppingCriteriaList([stop])
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    partial_message = ""
    for new_token in streamer:
        partial_message += new_token
        if '</s>' in partial_message:  # Breaking the loop if the stop token is generated.
            break
        yield partial_message


In [ ]:
prediction = predict("hello",[])

In [ ]:
for result in prediction:
    print(result)

In [1]:
pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/metal


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/metal
Note: you may need to restart the kernel to use updated packages.


In [2]:
from transformers import AutoTokenizer, AutoModel

# 加载分词器和模型
tokenizer = AutoTokenizer.from_pretrained("/Users/tianlimin/chatglm3-6b", trust_remote_code=True)
model = AutoModel.from_pretrained("/Users/tianlimin/chatglm3-6b", trust_remote_code=True)

# 检查MPS支持并将模型转移到MPS
if torch.backends.mps.is_available():
    device = torch.device("mps")
    model = model.to(device).half()  # 使用半精度，因为MPS通常支持半精度运算
else:
    print("MPS not available on this device. Falling back to CPU.")
    device = torch.device("cpu")
    model = model.to(device)

model = model.eval()

# 运行模型
response, history = model.chat(tokenizer, "你好", history=[])
print(response)
response, history = model.chat(tokenizer, "晚上睡不着应该怎么办", history=history)
print(response)


/Users/tianlimin/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Setting eos_token is not supported, use the default one.
Setting pad_token is not supported, use the default one.
Setting unk_token is not supported, use the default one.
Loading checkpoint shards: 100%|██████████████████| 7/7 [00:08<00:00,  1.17s/it]


NameError: name 'torch' is not defined